In [2]:
import kagglehub
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [3]:
# Download dataset
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

# Load dataset
csv_path = os.path.join(path, "IMDB Dataset.csv")
df = pd.read_csv(csv_path)

100%|██████████| 25.7M/25.7M [00:02<00:00, 12.1MB/s]

Extracting files...


In [5]:
# Encode labels
label_map = {'positive': 1, 'negative': 0}
df['sentiment'] = df['sentiment'].map(label_map)

# Preprocess text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post', truncating='post')

In [6]:
# Split data
train_size = int(len(df) * 0.8)
x_train, x_test = padded_sequences[:train_size], padded_sequences[train_size:]
y_train, y_test = df['sentiment'][:train_size], df['sentiment'][train_size:]

In [7]:
# Build model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
# Train model
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.5280 - loss: 0.6889 - val_accuracy: 0.5007 - val_loss: 0.6915
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.5046 - loss: 0.6928 - val_accuracy: 0.5451 - val_loss: 0.6875
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.5197 - loss: 0.6914 - val_accuracy: 0.5511 - val_loss: 0.6852
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.5812 - loss: 0.6644 - val_accuracy: 0.7721 - val_loss: 0.5287
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - accuracy: 0.7643 - loss: 0.5200 - val_accuracy: 0.8454 - val_loss: 0.3660


In [9]:
# Evaluate model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Loss: {loss:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8471 - loss: 0.3634
Test Accuracy: 84.54%
Test Loss: 0.3660


In [21]:
# Test with custom sentence
def predict_sentiment(sentence):
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=200, padding='post', truncating='post')
    prediction = model.predict(padded_sequence)[0][0]
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    print(f"Predicted Sentiment: {sentiment} ({prediction:.4f})")




######
predict_sentiment("The film was a complete disaster, I regret watching it.")
# Expected: Negative
predict_sentiment("why did they not follow the book i am really sad and disappointed i was so looking forward to seeing this movie however if you have read the book maybe recently it might be very difficult to remain objective my wife had not read the book and she loved the movie br br reasons for the disappointment are 1 <OOV> involvement gone with the wind such a shame there is a very small part at the start where the <OOV> is created but even that does not stick to the facts why not the fact that <OOV> father was burned with the first <OOV> brand which is how <OOV> got involved would have been a perfect start to the movie 2 story line between the deceased pope and <OOV> completely gone this completely <OOV> up the motive for the stealing of the <OOV> 3 story line between <OOV> and <OOV> <OOV> completely non existent br br all in all too <OOV> storyline and cannot understand that dan brown allowed them to put his name against it maybe i should <OOV> this film in 10 years time when i cannot remember the excellent book anymore fat chance on forgetting the book")
# Expected: Negative
predict_sentiment("Recap: Something mysterious transmits radio signals from Antarctica. Julian Rome, a former SETI worker, is brought in. But the plot has too many holes—Washington somehow knows too much, the alien pod transmits in English, and the ending is too open-ended. The movie is a little entertaining, but too much effort is needed to fill in the plot gaps. 3/10.")
# Expected: Negative
#######
predict_sentiment("This movie was absolutely fantastic, I loved every moment!")
# Expected: Positive
predict_sentiment("Watched this last night and was amazed by the heartfelt story, excellent character development, and good vibes from the acting. The story follows an ordinary man who takes dance lessons and learns about himself and others. The film is lighthearted and funny, yet dignified. Highly recommended for families and teens.")
# Expected: Positive
predict_sentiment("this is a very beautiful and almost <OOV> film there is hardly any dialogue in it apart from the narration and the scenery and music compliment each other perfectly i didn't at first connect the red hair of the girl and the fox until it was pointed out to me by a friend who also has red hair it is almost an old fashioned type of children's films saying that children nowadays prefer animations like <OOV> or toy story etc but i feel that young people should be introduced more to the beauty and wonder of nature which this film certainly does maybe not the best ever film of its type but certainly an excellent and relaxing view for all ages not just children")
# Expected: Positive
#####


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted Sentiment: Negative (0.1419)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Sentiment: Negative (0.3636)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Sentiment: Negative (0.0654)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Sentiment: Positive (0.7627)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Sentiment: Positive (0.9434)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Predicted Sentiment: Positive (0.9436)
